Types of methods to be used:
1. Cross-Correlation Analysis
2. Granger Causality Test
3. VAR Model
4. Impulse Response Analysis
5. Lagged Regression Analysis 

In [1]:
import MetaTrader5 as mt5
from datetime import datetime
import pytz
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import warnings
warnings.filterwarnings("ignore")
mt5.initialize()
account=51127988
password="Aar2frM7"
server = 'ICMarkets-Demo'

In [2]:
def get_rates(pair1, tf, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, tf, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    pair1 = pair1.set_index(pair1['time'])
    pair1 = pair1.drop(columns = ['time','tick_volume', 'spread', 'real_volume'])
    return pair1

## Hypothesis 
Currency prices whose sessions open prior to other sessions can be a leading indicator of prices for currencies sessions not yet opened, which can be found through interdependent and time lagged analysis. For example, if the Sydney session open in the first hour returns +10 pips, USDJPY is expected to return -10 pips in the first hour. 

## Method

Gather data on the first 5min to 60min bars of each open, starting with Sydney. Will base currencies against the USD Markets open in the following order:

- Sydney: 7PM (AUDUSD)
- Tokyo: 8PM (USDJPY)
- Frankfurt: 1PM (EURUSD)
- London: 2PM (GBPUSD)
- New York: 3PM (NIL)

Run a Pearson Correlation and a Granger Causality method on the gathered data and the respective currencies.



In [5]:
#Starting with AUDUSD, Hourly Timeframe 
AUDUSD = get_rates('AUDUSD.a', mt5.TIMEFRAME_H1, 50000)
syd_opens = AUDUSD.between_time('19:00','19:59')
syd_opens['Syd Pip Return'] = 10000 * (syd_opens['open'] - syd_opens['close'])
USDJPY = get_rates('USDJPY.a', mt5.TIMEFRAME_H1, 50000)
tokyo_opens = USDJPY.between_time('20:00', '20:59')
tokyo_opens

,open,high,low,close
time,,,,
2015-06-03 20:00:00,124.371,124.423,124.317,124.400
2015-06-04 20:00:00,124.516,124.535,124.346,124.348
2015-06-05 20:00:00,125.516,125.617,125.459,125.593
2015-06-08 20:00:00,125.055,125.075,124.688,124.744
2015-06-09 20:00:00,124.288,124.427,124.252,124.403
...,...,...,...,...
2023-06-08 20:00:00,138.923,138.980,138.898,138.940
2023-06-09 20:00:00,139.362,139.368,139.292,139.325
2023-06-12 20:00:00,139.714,139.768,139.576,139.637


In [52]:
tokyo_opens['Tko Pip Return'] = (tokyo_opens['open'] - tokyo_opens['close'])

In [55]:
pearson_corr = np.corrcoef(syd_opens['Syd Pip Return'].tail(5), tokyo_opens['Tko Pip Return'].tail(5)) 
pearson_corr

array([[1.       , 0.6430547],
       [0.6430547, 1.       ]])